# Evaluation Tests

In [ ]:
# IMPORTS
import torch as t
from torch.nn import ReLU, SELU, LeakyReLU
import numpy as np
from random import random, sample
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from parse_dataset import *
from icnn import ICNN, ICNNBN, gradient_step_action, diff_params, loss_beyond_RMAX, clip_gradients, get_q_target, update_parameters_lag
from copy import deepcopy
from utils import variable, moving_avg
from replay_buffer import PrioritizedReplayBuffer
from IPython.display import display, clear_output
from time import time
import evaluation
import importlib

## Load and parse Data

In [ ]:
data = pd.read_csv('Sepsis_imp.csv')
replace_absurd_temperatures(data)
data = drop_patients_with_absurd_weights(data)
data = drop_patients_with_unrealistic_HR_or_BP(data)
data = add_relative_time_column(data)
data = drop_patient_with_negative_input(data)
add_small_quantities(data)
create_action_column(data)
add_log_actions(data)



In [ ]:
# Build transitions array
# limit_patients = True
log_scaler = StandardScaler()
scaler = StandardScaler()
action_scaler = StandardScaler()
train_idx, test_idx = split_train_test_idx(data)

# scale on train data only
scaler.fit(data.loc[data.icustayid.isin(train_idx)][numerical_columns_not_to_be_logged])
log_scaler.fit(np.log(data.loc[data.icustayid.isin(train_idx)][numerical_columns_to_be_logged]))
action_scaler.fit(data.loc[data.icustayid.isin(train_idx)][log_action_cols])


limit_patients = True
num_patients = 100
if limit_patients:
    train_idx = data.icustayid.unique()[:num_patients]
print('limit_patients:',limit_patients,'len(train_idx):',len(train_idx))
transitions_train = transition_iterator(data, idx=train_idx, scaler=scaler, log_scaler=log_scaler,  action_scaler=action_scaler, RMAX=15, log_action=True)
print('len(transitions_train):',len(transitions_train))


In [ ]:
transitions_dict = {k: {
    "s": values[0],
    "a": values[1],
    "r": values[2],
    "s'": values[3]   
}
 for k, values in enumerate(transitions_train)
}

print('len(transitions_dict):',len(transitions_dict))

In [ ]:
# Compute extremums of action space
min0 = min([d['a'][0] for k, d in transitions_dict.items()])
max0 = max([d['a'][0] for k, d in transitions_dict.items()])
min1 = min([d['a'][1] for k, d in transitions_dict.items()])
max1 = max([d['a'][1] for k, d in transitions_dict.items()])

print('min0, max0, min1, max1:', min0, max0, min1, max1)

# Also save min and max of rewards
rmin = min([d['r'] for k, d in transitions_dict.items()])
rmax = max([d['r'] for k, d in transitions_dict.items()])
print('rmin, rmax:', rmin,rmax)

## Build Network and PER Buffer

In [ ]:
Q_select = ICNNBN(3, 20, 50, activation=SELU())
Q_eval = deepcopy(Q_select)

In [ ]:
# CREATE PRIORITIZED ER EXPERIENCE BUFFER
memory = PrioritizedReplayBuffer(len(transitions_dict), .6)
n_initial_0 = 500  # this is the initial number of transitions having reward 0
n_initial_15 = 1000  # this is the initial number of transitions having reward 15
n_initial_m15 = 5000  # this is the initial number of transitions having reward -15
count_15 = 0
count_0 = 0
count_m15 = 0

saved = set()
for idx, tr in shuffle(list(transitions_dict.items())):
    save = False
    if tr['r'] == 0 and count_0 < n_initial_0:
        count_0 += 1
        save = True
    if tr['r'] == -15 and count_m15 < n_initial_m15:
        count_m15 += 1
        save = True
    if tr['r'] == 15 and count_15 < n_initial_15:
        count_15 += 1
        save = True
    
    if save:
        saved.add(idx)
        # check if state is terminal
        if tr["s'"] is not None:
            s_ = tr["s'"]
            done = False
        else:
            s_ = np.array(50*[np.nan])
            done = True
        s = tr['s']
        a = tr['a']
        r = np.array([tr['r']])

        transition = (s,a,r,s_,done)
        memory.add(*transition)

In [ ]:
not_present_yet = shuffle([k for k in transitions_dict if k not in saved])

## Train network

In [ ]:
def clip(x):
    """
    Clip val
    ues below -RMAX or above RMAX
    Might be useful when the target goes beyond
    """
    return x*((x>=-RMAX).float())*((x<=RMAX).float()) + RMAX*(x>RMAX).float() - RMAX*(x<-RMAX).float()

In [ ]:
from time import time
plt.rcParams['figure.figsize'] = (20,20)

# RL parameters
global_step = -1
EPOCHS = 100
gamma = .99
max_steps_beta = 100000
beta0 = .4
beta = lambda t: (t<max_steps_beta)*(beta0 + t*(1-beta0)/max_steps_beta) + (t>=max_steps_beta)*1.
RMAX = 15
c = 1e4
T_UPDATE = 200  # parameter to copy the weights periodically

# OPTIMIZER PARAMETERS
batch_size = 32
learning_rate = 1e-3  # can have higher than default value if use BatchNorm
optimizer = t.optim.Adam(Q_select.parameters(), lr=learning_rate)
max_steps = int(len(data) / batch_size)
max_steps_a = 8

# monitoring
losses = []
td_errors = []
test_td_errors = []
test_average_q_pred = []
test_max_q_pred = []
test_min_q_pred = []
average_q_pred = []
average_q_target = []
max_q_pred = []
min_q_pred = []
max_q_target = []
min_q_target = []
average_vaso = []
max_vaso = []
min_vaso = []
average_fluid = []
max_fluid = []
min_fluid = []


# TRAIN
for _ in range(EPOCHS):
    t0 = time()
    for step in range(max_steps):
        global_step += 1

        # Add transition in memory
        idx = not_present_yet.pop()
        tr = transitions_dict[idx]
        if tr["s'"] is not None:
            s_ = tr["s'"]
            done = False
        else:
            s_ = np.array(50*[np.nan])
            done = True
        s = tr['s']
        a = tr['a']
        r = np.array([tr['r']])
        transition = (s,a,r,s_,done)
        memory.add(*transition)
        
        # Sample batch
        s,a,r,s_,done,w,idx = memory.sample(batch_size, beta(global_step))
        states = variable(s)
        actions = variable(a)
        rewards = variable(r)
        next_states = variable(s_)
        weights = variable(w).squeeze()
        
        # Init grad (set all of them to zero)
        optimizer.zero_grad()

        # Compute loss
        pred = Q_select.forward(states, actions).squeeze()  # Q(s, a)
        Q_select.eval()
        target, max_actions = get_q_target(Q_select, Q_eval, next_states, rewards, min0, max0, min1, max1, gamma=gamma, max_steps_a=max_steps_a, return_max_actions=True)  # max_a' Q(s', a)
        Q_select.train()
        target = clip(target.squeeze())
        loss = (pred-target)
        loss = loss**2
        loss = loss*weights  # multiplication by the PER coefficients
        loss = t.mean(loss) + c*t.mean(loss_beyond_RMAX(pred, RMAX))
        
        # Update priorities
        td_error = t.abs(pred - target) + 1e-3
        memory.update_priorities(idx, td_error.data.numpy())
        
        # Monitoring
        losses.append(loss.data.numpy()[0])
        td_errors.append(td_error.data.numpy().mean())
        average_q_pred.append(t.mean(pred).squeeze().data.numpy()[0])
        max_q_pred.append(t.max(pred).squeeze().data.numpy()[0])
        min_q_pred.append(t.min(pred).squeeze().data.numpy()[0])
        average_q_target.append(t.mean(target).squeeze().data.numpy()[0])
        max_q_target.append(t.max(target).squeeze().data.numpy()[0])
        min_q_target.append(t.min(target).squeeze().data.numpy()[0])
        min_a = np.min(max_actions.data.numpy(), 0).squeeze()
        max_a = np.max(max_actions.data.numpy(), 0).squeeze()
        mean_a = np.mean(max_actions.data.numpy(), 0).squeeze()
        min_vaso.append(min_a[0])
        max_vaso.append(max_a[0])
        average_vaso.append(mean_a[0])
        min_fluid.append(min_a[1])
        max_fluid.append(max_a[1])
        average_fluid.append(mean_a[1])
        
        # Compute gradients and update weights of the selection network
        loss.backward()
        clip_gradients(Q_select, 10)
        optimizer.step()
        
        # Keep weights positive so that the Q_select stays concave
        Q_select.proj()
        
        # update parameters of the evaluation network. Its weights lag behind
        if global_step % T_UPDATE == 0:
            Q_eval = deepcopy(Q_select)
            Q_eval.eval()

        # PLOT
        if step % 250 == 0:
            clear_output(wait=True)
            
            fig, axes = plt.subplots(3, 2)
            # td error
            axes[0,0].plot(moving_avg(losses))
            axes[0,1].plot(moving_avg(td_errors))
            # Q values (target and pred)
            axes[1,0].plot(moving_avg(average_q_pred), label='avg pred', c='darkblue')
            axes[1,0].plot(moving_avg(min_q_pred), label='min pred', c='darkblue', alpha=.8,linestyle=':')
            axes[1,0].plot(moving_avg(max_q_pred), label='max pred', c='darkblue', alpha=.8,linestyle=':')
            axes[1,0].plot(moving_avg(average_q_target), label='avg target', c='crimson')
            axes[1,0].plot(moving_avg(min_q_target), label='min target', c='crimson', alpha=.8,linestyle=':')
            axes[1,0].plot(moving_avg(max_q_target), label='max target', c='crimson', alpha=.8,linestyle=':')
            axes[1,1].plot(moving_avg(average_q_pred[200:]), label='avg pred', c='darkblue')
            axes[1,1].plot(moving_avg(min_q_pred[200:]), label='min pred', c='darkblue', alpha=.8,linestyle=':')
            axes[1,1].plot(moving_avg(max_q_pred[200:]), label='max pred', c='darkblue', alpha=.8,linestyle=':')
            axes[1,1].plot(moving_avg(average_q_target[200:]), label='avg target', c='crimson')
            axes[1,1].plot(moving_avg(min_q_target[200:]), label='min target', c='crimson', alpha=.8,linestyle=':')
            axes[1,1].plot(moving_avg(max_q_target[200:]), label='max target', c='crimson', alpha=.8,linestyle=':')
            axes[1,1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
            # actions (vaso and fluid)
            axes[2,0].plot(moving_avg(average_fluid), label='avg fluid', c='darkblue')
            axes[2,0].plot(moving_avg(min_fluid), label='min fluid', c='darkblue', alpha=.8,linestyle=':')
            axes[2,0].plot(moving_avg(max_fluid), label='max fluid', c='darkblue', alpha=.8,linestyle=':')
            axes[2,0].plot(moving_avg(average_vaso), label='avg vaso', c='crimson')
            axes[2,0].plot(moving_avg(min_vaso), label='min vaso', c='crimson', alpha=.8,linestyle=':')
            axes[2,0].plot(moving_avg(max_vaso), label='max vaso', c='crimson', alpha=.8,linestyle=':')
            axes[2,1].plot(moving_avg(average_fluid[200:]), label='avg fluid', c='darkblue')
            axes[2,1].plot(moving_avg(min_fluid[200:]), label='min fluid', c='darkblue', alpha=.8,linestyle=':')
            axes[2,1].plot(moving_avg(max_fluid[200:]), label='max fluid', c='darkblue', alpha=.8,linestyle=':')
            axes[2,1].plot(moving_avg(average_vaso[200:]), label='avg vaso', c='crimson')
            axes[2,1].plot(moving_avg(min_vaso[200:]), label='min vaso', c='crimson', alpha=.8,linestyle=':')
            axes[2,1].plot(moving_avg(max_vaso[200:]), label='max vaso', c='crimson', alpha=.8,linestyle=':')
            axes[2,1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
            plt.tight_layout()
            plt.show()
            
            print('Epoch %s\n%s steps in this epoch\n%s steps/s' % (str(_), str(step), str(step/(time()-t0))))

## Evaluate with Function A (Omer)

In [ ]:

# Build parameters for the function
discrete_transitions = np.load('trajectories.npy')
extremums_of_action_space = (min0,max0,min1,max1)
fence_posts = [ i for i in range(1,len(discrete_transitions)) if discrete_transitions[i][0] != discrete_transitions[i-1][0] ]
fence_posts = fence_posts[:num_patients-1]
print(discrete_transitions[:20])
print(fence_posts[:20])
states_sequence = [t[1] for t in discrete_transitions if (t[0] in train_idx)]
actions_sequence = [t[2] for t in discrete_transitions if (t[0] in train_idx)]
rewards_sequence = [v[2] for v in transitions_train]
# print(transitions_train[:20])
# rewards_sequence = [t[3] for t in discrete_transitions if (t[0] in train_idx)]
trans_as_tuples = [(s,a,r,s_) for k,s,a,r,s_ in discrete_transitions]
# print(rewards_sequence[:20])

In [ ]:
importlib.reload(evaluation)

DR_estim, indiv_estims = evaluation.eval_ICNN_WDR_Omer(Q_select, extremums_of_action_space, states_sequence, actions_sequence, rewards_sequence, fence_posts, trans_as_tuples, gamma)

print('type of DR_estim:', type(DR_estim))
print('DR_estim:', DR_estim)


In [ ]:
print(indiv_estims)
print(DR_estim)

## Evaluate with Function B (HW3)

In [ ]:

# Compute D, which is the same as transitions_train but as a list of lists where every element is an episode

discrete_transitions = np.load('trajectories.npy')
previous_id = discrete_transitions[0,0]
D_train=[]
episode = []
for i,trans in tqdm(enumerate(discrete_transitions)):
    if trans[0] in train_idx:
        if trans[0] != previous_id:
#             print('Appending ',len(episode),'-step episode from patient',previous_id, 'to D')
            previous_id = trans[0]
            D_train.append(deepcopy(episode))
            episode = []   
        # We're using rewards -15 and 15, so we need to correct the rewards that come with the
        # file (was created with rewards -10 and 20)
        if trans[3]:
            r = trans[3]-5
        else: r = trans[3]
        episode.append((trans[1],trans[2], r, trans[4]))    

print('Final length of D_train:',len(D_train)) 
print('First few episodes of D_train:',D_train[:2])

## TODO: CHANGE TRANSITIONS SO THAT THE LAST STATE OF AN EPISODE IS STATE 751 IF ALIVE, 752 IF DEAD

In [ ]:
importlib.reload(evaluation)

# DR estimator
extremums_of_action_space = (min0,max0,min1,max1)
DR = evaluation.eval_ICNN_DR_HW3(Q_select, extremums_of_action_space, D_train, gamma, rmin, rmax)
print('DR:', DR)
plt.plot(DR)
plt.title('DR value at step', step)
plt.show()

## Evaluate with function C (Continuous WDR implementation)

In [ ]:
# Build parameters for the function
discrete_transitions = np.load('trajectories.npy')
extremums_of_action_space = (min0,max0,min1,max1)
fence_posts = [ i for i in range(1,len(discrete_transitions)) if discrete_transitions[i][0] != discrete_transitions[i-1][0] ]
fence_posts = fence_posts[:num_patients-1]
print('discrete_transitions[:5]',discrete_transitions[:5])
print('fence_posts[:5]',fence_posts[:5], 'len(fence_posts):', len(fence_posts))
cont_states_sequence = [variable(t[0]) for t in transitions_train]
cont_actions_sequence = [variable(t[1]) for t in transitions_train]
discrete_states = [t[1] for t in discrete_transitions if (t[0] in train_idx)]
# discrete_actions_2 = [t[2] for t in discrete_transitions if (t[0] in train_idx)]
discrete_actions = [t[1]['action'] for t in data.iterrows() if (t[1]['icustayid'] in train_idx)]
rewards_sequence = [v[2] for v in transitions_train]
print('cont_states_sequence[12:20]',cont_states_sequence[12:20])
print('discrete_states[12:20]',discrete_states[12:20])
# print('discrete_actions_2[:20]',discrete_actions_2[:20])
assert(len(cont_states_sequence) == len(discrete_states))

quantiles_fluid, quantiles_vaso = compute_action_quantiles(data)

print(quantiles_fluid, quantiles_vaso)

In [ ]:
importlib.reload(evaluation)

DR_estim, indiv_estim = evaluation.eval_ICNN_WDR_continuous(Q_select, extremums_of_action_space, cont_states_sequence, cont_actions_sequence, rewards_sequence, 
                             quantiles_fluid, quantiles_vaso, fence_posts, gamma)


print('type of DR_estim:', type(DR_estim))
print('DR_estim:', DR_estim)

In [ ]:
np.save('Indiv_WDR_99_patients.npy', np.array(indiv_estim))

In [ ]:
loaded_indiv_estims = np.load('Indiv_WDR_99_patients.npy')
print(loaded_indiv_estims[98])

## New approach for computing integral: network for predicting int(Q)

In [ ]:
# Network for predicting the integral of the Q function

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import icnn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
importlib.reload(icnn)
import torch.optim as optim


class Net(nn.Module):

    def __init__(self, input_dim, n_layers, hidden_dim, activation = ReLU()):
        super(Net, self).__init__()
        self.input_dim = input_dim
        self.n_layers = n_layers
        self.activation = activation
        self.hidden_dim = hidden_dim
        
        setattr(self, 'fcin', nn.Linear(input_dim, hidden_dim))
        for l in range(self.n_layers):
            output_dim = (l < n_layers - 1) * hidden_dim + (l == n_layers - 1) * 1
            setattr(self, 'fc'+str(l), nn.Linear(hidden_dim, output_dim))
            
        self.initialize_weights_selu()
        
    def initialize_weights_selu(self):
        shape = self.fcin.weight.data.numpy().shape
        self.fcin.weight.data = t.from_numpy(np.random.normal(0, 1 / np.sqrt(shape[0]), shape)).float()
        for l in range(self.n_layers):
            fc = getattr(self, 'fc'+str(l))
            shape = fc.weight.data.numpy().shape
            fc.weight.data = t.from_numpy(np.random.normal(0, 1 / np.sqrt(shape[0]), shape)).float()
        
    def forward(self, z):
#         print("Incoming z:", z)
        z_ = self.fcin(z)
        print("Output z_ after first layer:", z_)
        z_ = self.activation(z_)
        print("Output z_ after first activation:", z_)
        for l in range(self.n_layers):
            fc = getattr(self, 'fc' + str(l))
            z_ = fc(z_)
            print("Output z_ after layer",l, z_)
            z_ = self.activation(z_)
        return z_

net = Net(len(transitions_train[1][0]),3, 20)
print(net)

params = list(net.parameters())
print(params)

In [ ]:
# Building training and test sets

states_data = [trans[0] for trans in transitions_train]
integrals = np.zeros(len(states_data))
for i,s in enumerate(states_data):
    integrals[i]=icnn.compute_integral(Q_select, variable(s), min0, max0,min1,max1).data.numpy()
print(integrals[:10])
x_train, x_test, y_train, y_test = train_test_split(states_data, integrals, test_size=0.2)    
print('len(x_train)',len(x_train))
print('len(y_train)',len(y_train))
print('len(x_test)',len(x_test))
# print('x_train[:5]',x_train[:5])
# print('y_train[:5]',y_train[:5])

## Training network

In [ ]:

# create your optimizer
learning_rate = 1e-4
optimizer = t.optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

training_epochs = 5

for e in range(training_epochs):
    for i,s in enumerate(x_train):
        optimizer.zero_grad()
        print('input to net:',variable(s))
        int_pred = net.forward(variable(s))
        target = variable(y_train[i])  
        net.zero_grad()
        loss = criterion(int_pred, target)
        loss.backward()
        optimizer.step()
#         print('pred,target:',int_pred,target)
#         print('loss',loss)
        if i%200 == 0:
            print('current loss:', loss.data.numpy())

## Testing network

In [ ]:
# Helper function
def calculate_accuracy(y_pred, y_true, print_= False, label=''):
    corr_labeled = sum(y_pred == y_true)
    accuracy = corr_labeled/len(y_pred)
    if print_:
        print(label,'Accuracy =', accuracy*100, '% (',corr_labeled,'correctly labeled /',len(y_pred),'observations in given set )')
    return accuracy, corr_labeled

# Getting final predictions
int_pred_train = net(variable(x_train))
int_pred_test = net(variable(x_test))

int_pred_train = int_pred_train.data.numpy().squeeze()
int_pred_test = int_pred_test.data.numpy().squeeze()

print(int_pred_train)
print(y_train)

# Print first 5 predictions vs first 5 trues:
print('First 5 preds and trues:', int_pred_train[:5],y_train[:5])

# Calculating R2 score
print('R2 Score train:', r2_score(y_train,int_pred_train))
print('R2 Score test:', r2_score(y_test,int_pred_test))